In [ ]:
import pandas as pd
import Memory_Collaborative_Filtering as mem
import sqlite3 as sql
import sklearn 
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import pickle
import sqlite3 as sql

In [ ]:
worm_path = 'C:/Users/arjun/Desktop/amazon.db'
freq_path = 'C:/Users/arjun/Desktop/frequency_rankings.db'
meta_path = 'C:/users/arjun/Desktop/test.csv'

In [ ]:
review_data = mem.import_standard_data(worm_path)

In [ ]:
top_product_counts = mem.import_product_frequency_data(freq_path)

In [ ]:
product_metadata = pd.read_csv(meta_path)
product_metadata.to_csv('C:/users/arjun/Desktop/product_metadata.csv')

In [ ]:
wrangled_review_data = review_data.drop_duplicates(['customer_id', 'product_id'])
wrangled_review_data['star_rating'] = wrangled_review_data['star_rating'].astype(int)
wrangled_review_data['review_length'] = wrangled_review_data['review_body'].str.count(' ') + 1
wrangled_review_data = wrangled_review_data.drop(['customer_id'], axis=1)
wrangled_review_data = wrangled_review_data.drop_duplicates()
wrangled_review_data = wrangled_review_data.reset_index(drop=True)

In [ ]:
final_product_data = pd.merge(wrangled_review_data, top_product_counts, left_on='product_id', right_on='product_id', how='inner')
final_product_data = pd.merge(final_product_data, product_metadata, left_on='product_id', right_on='product_id', how='left')
final_product_data = final_product_data[['product_id', 'star_rating', 'helpful_votes', 'review_length', 'counts', 'price', 'salesRank']]

In [ ]:
final_product_data = final_product_data.dropna()
final_product_data = final_product_data[['star_rating', 'helpful_votes', 'review_length', 'counts', 'price', 'salesRank']]

In [ ]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm = 'ball_tree').fit(final_product_data)

In [ ]:
def get_index_from_name(name, review_data):
    return review_data[review_data["product_title"]==name].index.tolist()[0]

In [ ]:
def print_similar_books(review_data, query=None,id=None, model=None):
    distances2, indices2 = model.kneighbors()
    if id:
        for id in indices2[id][1:]:
            print(review_data.iloc[id]["product_title"])
    if query:
        found_id = get_index_from_name(query, review_data)
        for id in indices2[found_id][1:]:
            print(review_data.iloc[id]["product_title"])

In [ ]:
test_recs = wrangled_review_data.head(5)
print(test_recs)

In [ ]:
for book in test_recs['product_title']:
    print(book + " recommendations:")
    print_similar_books(wrangled_review_data, query=book, model=nbrs)